# Lab 1: Azure AI Foundry 리소스 배포하기 (Deploy Azure Resources)

## 개요 (Overview)

이 노트북에서는 Azure AI Foundry 기반 멀티 에이전트 시스템을 구축하기 위한 Azure 리소스를 배포합니다.

### 배포할 리소스 (Resources to Deploy)

- Azure AI Foundry Project
- Azure AI Search (RAG 지식 베이스용)
- Azure Container Apps Environment (에이전트 호스팅용)
- Azure Container Registry (이미지 저장용)
- Azure Key Vault (비밀 관리용)
- Azure Storage Account (데이터 저장용)
- Azure OpenAI Service (GPT-4 모델용)

### 아키텍처 (Architecture)

```
┌────────────────────────────────────────────────────────────┐
│                 Multi-Agent System                         │
│                                                            │
│  ┌─────────────────────────────────────────────┐          │
│  │          Main Agent                         │          │
│  │  (Task Analysis & Routing)                  │          │
│  └────────────┬────────────────┬────────────────┘          │
│               │                │                           │
│       ┌───────▼──────┐  ┌──────▼──────────┐               │
│       │  Tool Agent  │  │  Research       │               │
│       │  (MCP)       │  │  Agent (RAG)    │               │
│       └──────┬───────┘  └────────┬────────┘               │
│              │                   │                         │
│       ┌──────▼───────┐    ┌──────▼─────────┐              │
│       │  MCP Server  │    │  Azure AI      │              │
│       │  (Weather)   │    │  Search (RAG)  │              │
│       └──────────────┘    └────────────────┘              │
└────────────────────────────────────────────────────────────┘
```

### 학습 목표 (Learning Objectives)

1. ✅ Azure Developer CLI (azd)를 사용한 인프라 배포
2. ✅ Bicep 템플릿 기반 Infrastructure as Code 이해
3. ✅ Azure AI Foundry 리소스 구조 파악
4. ✅ 배포된 리소스 확인 및 연결 정보 수집

## 1. 사전 요구 사항 확인 (Prerequisites Check)

다음 도구들이 설치되어 있는지 확인합니다:

- Python 3.9 이상
- Azure CLI
- Azure Developer CLI (azd)
- Docker (Container Apps 배포 시 필요)

In [1]:
import sys, subprocess, os
import platform

# 운영체제에 따라 PATH 설정 (macOS, Linux, Codespaces 모두 지원)
system = platform.system()
if system == 'Darwin':  # macOS
    # Homebrew 경로 추가 (Intel & Apple Silicon)
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    # 일반적인 Linux 바이너리 경로
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("azd version", "Azure Developer CLI")
check("docker --version", "Docker")
print("="*50)

=== Prerequisites Check ===
✓ Python 3.13.7 (Darwin)
✓ Azure CLI
✓ Azure Developer CLI
✓ Docker


## 2. Azure 인증 (Azure Authentication)

Azure에 로그인하고 구독을 설정합니다. 브라우저가 열리고 인증을 진행합니다.

In [2]:
import subprocess, json

print("=== Azure Authentication ===\n🔐 Browser will open for authentication...\n")

az = subprocess.run("az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3", shell=True, capture_output=True, text=True)
print(f"{'✅' if az.returncode == 0 else '❌'} Azure CLI")

azd = subprocess.run("azd auth login", shell=True, capture_output=True, text=True)
print(f"{'✅' if azd.returncode == 0 else '❌'} Azure Developer CLI")

print("="*50)

=== Azure Authentication ===
🔐 Browser will open for authentication...

✅ Azure CLI
✅ Azure Developer CLI


In [3]:
import subprocess, json

print("=== Verifying Azure Authentication ===\n")

result = subprocess.run("az account show", shell=True, capture_output=True, text=True, timeout=5)

if result.returncode == 0:
    account = json.loads(result.stdout)
    print(f"✅ Authenticated as: {account['user']['name']}")
    print(f"   Subscription: {account['name']}")
else:
    print("❌ Not authenticated - Run previous cell")

print("="*50)

=== Verifying Azure Authentication ===

✅ Authenticated as: junwoojeong@microsoft.com
   Subscription: MCAPS-Hybrid-REQ-46855-2022-junwoojeong


## 3. 환경 변수 설정 (Environment Configuration)

Azure Developer CLI를 위한 환경을 설정합니다.

In [4]:
# 프로젝트 루트 디렉토리로 이동 (절대 경로 사용)
import os
from pathlib import Path

# 프로젝트 루트 디렉토리 (노트북이 루트에 위치)
project_root = Path.cwd()

os.chdir(project_root)
print(f"✓ Project root: {os.getcwd()}")

# 디렉토리 구조 확인
checks = [
    ('infra/main.bicep', 'Bicep templates'),
    ('azure.yaml', 'azure.yaml'),
    ('src/agents', 'Agent source code')
]

for path, name in checks:
    if os.path.exists(path):
        print(f"✓ {name} found")
    else:
        print(f"✗ {name} NOT found")
        
print("="*50)

✓ Project root: /Users/junwoojeong/GitHub/agentic-ai-labs
✓ Bicep templates found
✓ azure.yaml found
✗ Agent source code NOT found


In [5]:
import random, string, json

# 현재 Azure 구독 정보 가져오기
subscription_result = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if subscription_result.returncode != 0:
    print("❌ Not authenticated! Please run Cell 5 (Azure Authentication) first.")
    raise Exception("Azure authentication required")

account_info = json.loads(subscription_result.stdout)
subscription_id = account_info['id']
subscription_name = account_info['name']

print(f"📋 Using Subscription: {subscription_name}")
print(f"   ID: {subscription_id}\n")

# 환경 이름 생성
random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
environment_name = f"aiagent-{random_suffix}"
location = "eastus"

print(f"Environment Name: {environment_name}\nLocation: {location}\n")

# azd 환경 생성 및 구독 설정
subprocess.run(f"azd env new {environment_name} --subscription {subscription_id} --no-prompt", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_SUBSCRIPTION_ID {subscription_id}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_LOCATION {location}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_ENV_NAME {environment_name}", shell=True, capture_output=True)

print("✓ Environment configured with subscription")

📋 Using Subscription: MCAPS-Hybrid-REQ-46855-2022-junwoojeong
   ID: 49a89096-a0ae-4e59-816b-dcb0a6fe9168

Environment Name: aiagent-ciid4s
Location: eastus

✓ Environment configured with subscription


In [6]:
principal_result = subprocess.run("az ad signed-in-user show --query id -o tsv", shell=True, capture_output=True, text=True)

if principal_result.returncode == 0:
    principal_id = principal_result.stdout.strip()
    print(f"Principal ID: {principal_id}\n")
    
    subprocess.run(f"azd env set AZURE_PRINCIPAL_ID {principal_id}", shell=True, capture_output=True)
    subprocess.run(f"azd env set principalId {principal_id}", shell=True, capture_output=True)
    subprocess.run(f"azd env set environmentName {environment_name}", shell=True, capture_output=True)
    subprocess.run(f"azd env set location {location}", shell=True, capture_output=True)
    
    print("✓ All parameters configured")
else:
    print("❌ Failed to get Principal ID")

Principal ID: c9e4bb1f-c7b3-4187-b68d-890d3cbb6f12

✓ All parameters configured


## 4. 인프라 배포 (Deploy Infrastructure)

**⚠️ 중요 알림:**
- Azure OpenAI, AI Foundry 등의 리소스가 생성됩니다
- 예상 비용: 시간당 약 $5-10 (사용량에 따라 다름)
- **이 단계에서는 인프라만 생성하고, Container Apps는 생성하지 않습니다**

### 배포되는 리소스

1. **Resource Group** - 모든 리소스를 포함하는 그룹
2. **AI Foundry Project** - 에이전트 프로젝트 워크스페이스
3. **Azure OpenAI** - GPT-4 모델 (gpt-4, gpt-4o, text-embedding-3-large)
4. **Azure AI Search** - RAG를 위한 지식 베이스
5. **Container Apps Environment** - 에이전트 호스팅 환경 (빈 환경만 생성)
6. **Container Registry** - 도커 이미지 저장소
7. **Key Vault** - 비밀 및 키 관리
8. **Storage Account** - 데이터 저장소
9. **Log Analytics Workspace** - 로그 및 모니터링

### 배포하지 않는 리소스

- **Container Apps (MCP Server)** - Notebook 03에서 이미지와 함께 생성

**📝 노트:**
- Container Apps는 **Notebook 03**에서 `azd deploy` 실행 시 Docker 이미지와 함께 생성됩니다
- 지금은 이미지 pull 없이 빠르게 인프라만 구성합니다 (약 3-5분 소요)

In [7]:
print("🚀 Starting Azure infrastructure deployment...")
print(f"Environment: {environment_name}")
print(f"Subscription: {subscription_id}")
print("⏱️  This will take approximately 3-5 minutes (no Container Apps)\n" + "="*50)

result = subprocess.run(
    f"azd provision --no-prompt --environment {environment_name}",
    shell=True,
    text=True,
    env={**os.environ, 'AZURE_SUBSCRIPTION_ID': subscription_id}
)

print("\n" + "="*50)
if result.returncode == 0:
    print("✅ Infrastructure deployment completed successfully!")
    print("\n📋 Created Resources:")
    print("   • AI Foundry Project & OpenAI Models")
    print("   • AI Search Service")
    print("   • Container Apps Environment (empty)")
    print("   • Container Registry")
    print("   • Key Vault, Storage, Monitoring")
    print("\n🚫 NOT Created (will be in Notebook 03):")
    print("   • Container Apps (MCP Server) - will be deployed with images")
    print("\n💡 Next Steps:")
    print("   1. Continue to Section 5: Verify deployed resources")
    print("   2. Notebook 02: Setup AI Search RAG")
    print("   3. Notebook 03: Build images & deploy Container Apps")
else:
    print(f"❌ Deployment failed with return code {result.returncode}")
    print("Check the error messages above for details.")

🚀 Starting Azure infrastructure deployment...
Environment: aiagent-ciid4s
Subscription: 49a89096-a0ae-4e59-816b-dcb0a6fe9168
⏱️  This will take approximately 3-5 minutes (no Container Apps)

Provisioning Azure resources (azd provision)
Provisioning Azure resources can take some time.

Initialize bicep provider
Reading subscription and location from environment...
Subscription: MCAPS-Hybrid-REQ-46855-2022-junwoojeong (49a89096-a0ae-4e59-816b-dcb0a6fe9168)
Location: East US

Creating a deployment plan
Comparing deployment state
Creating/Updating resources
  You can view detailed progress in the Azure Portal:
  https://portal.azure.com/#view/HubsExtension/DeploymentDetailsBlade/~/overview/id/%2Fsubscriptions%2F49a89096-a0ae-4e59-816b-dcb0a6fe9168%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Faiagent-ciid4s-1759408431

  (✓) Done: Resource group: rg-aiagent-ciid4s (6.619s)
  (✓) Done: Log Analytics workspace: log-pf3kkfblz2ryy (26.116s)
  (✓) Done: Application Insights: appi-pf3kkfblz2

## 5. 배포 결과 확인 (Verify Deployment)

배포된 리소스와 연결 정보를 확인합니다.

In [8]:
# azd 환경 변수 가져오기
env_result = subprocess.run(
    "azd env get-values",
    shell=True,
    capture_output=True,
    text=True
)

if env_result.returncode == 0:
    print("=== Deployed Resources Configuration ===\n")
    env_vars = {}
    for line in env_result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            # 따옴표 제거
            value = value.strip('"')
            env_vars[key] = value
            print(f"{key}: {value}")
    
    print("\n" + "="*50)
    print("✅ Environment variables loaded successfully")
else:
    print("❌ Failed to load environment variables")
    env_vars = {}

=== Deployed Resources Configuration ===

AGENT_MAIN_URL: 
AGENT_RESEARCHER_URL: 
AGENT_WRITER_URL: 
AZURE_AI_PROJECT_CONNECTION_STRING: https://aoai-pf3kkfblz2ryy.services.ai.azure.com/api/projects/proj-pf3kkfblz2ryy;subscription_id=49a89096-a0ae-4e59-816b-dcb0a6fe9168;resource_group=rg-aiagent-ciid4s
AZURE_CONTAINER_APPS_ENVIRONMENT_ID: /subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/providers/Microsoft.App/managedEnvironments/cae-pf3kkfblz2ryy
AZURE_CONTAINER_REGISTRY_ENDPOINT: crpf3kkfblz2ryy.azurecr.io
AZURE_CONTAINER_REGISTRY_NAME: crpf3kkfblz2ryy
AZURE_ENV_NAME: aiagent-ciid4s
AZURE_LOCATION: eastus
AZURE_OPENAI_ENDPOINT: https://aoai-pf3kkfblz2ryy.cognitiveservices.azure.com/
AZURE_PRINCIPAL_ID: c9e4bb1f-c7b3-4187-b68d-890d3cbb6f12
AZURE_RESOURCE_GROUP: rg-aiagent-ciid4s
AZURE_SEARCH_ENDPOINT: https://srch-pf3kkfblz2ryy.search.windows.net/
AZURE_SEARCH_SERVICE_NAME: srch-pf3kkfblz2ryy
AZURE_SUBSCRIPTION_ID: 49a89096-a0ae-4e59-816b-dcb0a6fe91

In [9]:
# 리소스 그룹의 리소스 목록 확인
if 'AZURE_RESOURCE_GROUP' in env_vars:
    rg_name = env_vars['AZURE_RESOURCE_GROUP']
    
    resources_result = subprocess.run(
        f'az resource list --resource-group {rg_name} --query "[].{{Name:name, Type:type, Location:location}}" --output table',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if resources_result.returncode == 0:
        print("=== Deployed Resources ===\n")
        print(resources_result.stdout)
    else:
        print("❌ Failed to list resources")
        print(f"Error: {resources_result.stderr}")
else:
    print("⚠️  Resource group name not found in environment variables")
    print("💡 Please run the previous cell (Cell 17) first to load environment variables.")

=== Deployed Resources ===

Name                                    Type                                                Location
--------------------------------------  --------------------------------------------------  ----------
log-pf3kkfblz2ryy                       Microsoft.OperationalInsights/workspaces            eastus
crpf3kkfblz2ryy                         Microsoft.ContainerRegistry/registries              eastus
stpf3kkfblz2ryy                         Microsoft.Storage/storageAccounts                   eastus
srch-pf3kkfblz2ryy                      Microsoft.Search/searchServices                     eastus
kv-pf3kkfblz2ryy                        Microsoft.KeyVault/vaults                           eastus
appi-pf3kkfblz2ryy                      Microsoft.Insights/components                       eastus
cae-pf3kkfblz2ryy                       Microsoft.App/managedEnvironments                   eastus
aoai-pf3kkfblz2ryy                      Microsoft.CognitiveServices/account

## 6. 주요 리소스 연결 정보 저장 (Save Configuration)

다음 노트북에서 사용할 연결 정보를 저장합니다.

In [ ]:
# 주요 연결 정보를 추출하여 config 파일로 저장
# 최소화된 환경변수만 사용 - AI Agent 실행에 필수적인 정보만 포함
config = {
    "resource_group": env_vars.get("AZURE_RESOURCE_GROUP", ""),
    "location": env_vars.get("AZURE_LOCATION", ""),
    "project_connection_string": env_vars.get("AZURE_AI_PROJECT_CONNECTION_STRING", ""),
    "search_endpoint": env_vars.get("AZURE_SEARCH_ENDPOINT", ""),
    "search_service_name": env_vars.get("AZURE_SEARCH_SERVICE_NAME", ""),
    "container_registry_endpoint": env_vars.get("AZURE_CONTAINER_REGISTRY_ENDPOINT", ""),
    "container_apps_environment_id": env_vars.get("AZURE_CONTAINER_APPS_ENVIRONMENT_ID", "")
}

# 프로젝트 루트에 config.json 저장
config_path = Path("config.json")

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("=== Configuration Saved (Minimized) ===")
print(f"Location: {config_path.absolute()}")
print("\nEssential Information:")
print(f"  - Resource Group: {config['resource_group']}")
print(f"  - Location: {config['location']}")
print(f"  - AI Project Connection: {'✓ Set' if config['project_connection_string'] else '✗ Missing'}")
print(f"  - Search Endpoint: {config['search_endpoint']}")
print(f"  - Container Registry: {config['container_registry_endpoint']}")
print("\n✅ Minimized configuration saved successfully!")
print("\n💡 Note: Project connection string contains all AI Foundry info (project, OpenAI, etc.)")

=== Configuration Saved (Minimized) ===
Location: /Users/junwoojeong/GitHub/agentic-ai-labs/config.json

Essential Information:
  - Resource Group: rg-aiagent-ciid4s
  - Location: eastus
  - AI Project Connection: ✓ Set
  - Search Endpoint: https://srch-pf3kkfblz2ryy.search.windows.net/
  - Container Registry: crpf3kkfblz2ryy.azurecr.io

✅ Minimized configuration saved successfully!

💡 Note: Project connection string contains all AI Foundry info (project, OpenAI, etc.)


## 7. Azure Portal에서 확인 (Verify in Azure Portal)

Azure Portal에서 배포된 리소스를 확인할 수 있습니다.

In [11]:
# Azure Portal 링크 생성
resource_group = env_vars.get("AZURE_RESOURCE_GROUP", "")
subscription_id = env_vars.get("AZURE_SUBSCRIPTION_ID", "")

if resource_group and subscription_id:
    portal_url = f"https://portal.azure.com/#@/resource/subscriptions/{subscription_id}/resourceGroups/{resource_group}/overview"
    print("✅ Azure Portal에서 리소스 그룹 확인:")
    print(f"🔗 {portal_url}")
    print("\n🎯 주요 리소스:")
    print("  • AI Project: 에이전트 프로젝트")
    print("  • Azure OpenAI: GPT-4 모델")
    print("  • AI Search: RAG 지식 베이스")
    print("  • Container Apps Environment: 에이전트 실행 환경")
else:
    print("⚠️ 환경 변수에서 리소스 그룹 정보를 찾을 수 없습니다.")

✅ Azure Portal에서 리소스 그룹 확인:
🔗 https://portal.azure.com/#@/resource/subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/rg-aiagent-ciid4s/overview

🎯 주요 리소스:
  • AI Project: 에이전트 프로젝트
  • Azure OpenAI: GPT-4 모델
  • AI Search: RAG 지식 베이스
  • Container Apps Environment: 에이전트 실행 환경


## 8. 배포 완료 및 요약 (Deployment Summary)

축하합니다! Azure AI Foundry 인프라 배포를 성공적으로 완료했습니다.

### 배포된 리소스 (Deployed Resources)

| 리소스 타입 | 용도 | 상태 |
|------------|------|------|
| AI Project | 에이전트 프로젝트 워크스페이스 | ✅ 생성됨 |
| Azure OpenAI | GPT-4, GPT-4o, Embeddings 모델 | ✅ 배포됨 |
| AI Search | RAG 지식 베이스 (벡터 검색) | ✅ 생성됨 |
| Container Registry | Docker 이미지 저장소 | ✅ 생성됨 |
| Container Apps Environment | 에이전트 호스팅 환경 | ✅ 생성됨 |
| Key Vault | 시크릿 및 연결 문자열 관리 | ✅ 생성됨 |
| Storage Account | 데이터 및 로그 저장 | ✅ 생성됨 |
| Log Analytics | 모니터링 및 로깅 | ✅ 생성됨 |
| Application Insights | 애플리케이션 성능 모니터링 | ✅ 생성됨 |

### 📝 중요 사항

**현재 완료된 작업:**
- ✅ Azure 인프라 리소스 생성 완료
- ✅ Container Apps Environment 준비 완료
- ✅ 역할 할당 및 권한 설정

**아직 완료되지 않은 작업:**
- ⏸️ Docker 이미지 빌드
- ⏸️ Container Registry에 이미지 푸시
- ⏸️ Container Apps 생성 및 배포

→ 이 작업들은 **Notebook 03**에서 진행됩니다.

### ✨ 개선 사항

**이전 방식 (문제점):**
- Container Apps를 placeholder 이미지로 먼저 생성
- 이미지 pull 시도로 20분 타임아웃 발생
- 불필요한 리소스 생성 및 삭제 반복

**현재 방식 (최적화):**
- Container Apps Environment만 생성 (3-5분 소요)
- Container Apps는 Notebook 03에서 실제 이미지와 함께 생성
- 깔끔하고 빠른 배포 프로세스

### 다음 단계 (Next Steps)

이제 다음 노트북으로 순서대로 진행하세요:

1. **Notebook 2: RAG 지식 베이스 구성** (`02_setup_ai_search_rag.ipynb`)
   - Azure AI Search 인덱스 생성
   - 샘플 데이터 임베딩 및 업로드
   - 하이브리드 검색 (벡터 + 키워드) 테스트

2. **Notebook 3: 에이전트 이미지 빌드 및 배포** (`03_azure_foundry_multi_agent.ipynb`)
   - MCP Server Docker 이미지 빌드
   - Container Registry에 이미지 푸시
   - Container Apps 생성 및 배포
   - 멀티 에이전트 시스템 테스트

## 9. 문제 해결 (Troubleshooting)

배포 중 문제가 발생한 경우 아래를 참고하세요.

### 일반적인 오류 및 해결 방법

#### 1. **할당량 초과 오류 (Quota exceeded)**
```
Error: Quota exceeded for resource type
```
**해결 방법:**
- Azure Portal에서 구독의 할당량 확인
- 다른 리전 선택: `azd env set AZURE_LOCATION <region>`
- 할당량 증가 요청: [Azure Support](https://portal.azure.com/#blade/Microsoft_Azure_Support/HelpAndSupportBlade)

#### 2. **리전에서 리소스 사용 불가**
```
Error: The resource type is not available in the location
```
**해결 방법:**
- Azure OpenAI 사용 가능 리전 확인: eastus, westus, northeurope, etc.
- 리전 변경: `azd env set AZURE_LOCATION eastus`

#### 3. **인증 오류**
```
Error: Authentication failed
```
**해결 방법:**
```bash
# Azure CLI 재인증
az login
az account set --subscription <subscription-id>

# Azure Developer CLI 재인증
azd auth login
```

#### 4. **Bicep 템플릿 오류**
```
Error: Deployment failed with validation errors
```
**해결 방법:**
- Bicep 파일 검증: `az bicep build --file infra/main.bicep`
- 로그 확인: Azure Portal → Resource Group → Deployments
- 상세 로그: `azd up --debug`

#### 5. **환경 변수 누락**
```
Error: Required environment variable not set
```
**해결 방법:**
```bash
# 필수 환경 변수 확인
azd env get-values

# Principal ID 재설정
azd env set AZURE_PRINCIPAL_ID $(az ad signed-in-user show --query id -o tsv)
```

### 배포 재시도

문제를 해결한 후 배포를 재시도할 수 있습니다:

```bash
# 기존 환경 삭제 (선택사항)
azd down --force --purge

# 새로 배포
azd up --no-prompt
```

### 로그 확인

배포 로그를 자세히 확인하려면:

```bash
# 상세 로그 출력
azd up --debug

# 특정 서비스 로그 확인 (배포 후)
azd logs --service main-agent
```

## 10. 리소스 정리 (Cleanup)

⚠️ **중요:** 실습이 완료되면 비용 발생을 방지하기 위해 리소스를 삭제하세요.

### 옵션 1: Azure Developer CLI로 전체 삭제 (권장)

모든 리소스를 한 번에 삭제합니다:

```python
# 주의: 이 명령은 모든 리소스를 영구적으로 삭제합니다!
# 실행 전 백업이 필요한 데이터가 있는지 확인하세요.
```

```bash
azd down --force --purge
```

**옵션 설명:**
- `--force`: 확인 없이 즉시 삭제
- `--purge`: Key Vault 등의 소프트 삭제된 리소스도 완전히 제거

### 옵션 2: Azure Portal에서 수동 삭제

1. [Azure Portal](https://portal.azure.com) 접속
2. 리소스 그룹으로 이동
3. "리소스 그룹 삭제" 클릭
4. 리소스 그룹 이름 입력하여 확인

### 옵션 3: Azure CLI로 삭제

```bash
# 리소스 그룹 삭제 (모든 리소스 포함)
az group delete --name <resource-group-name> --yes --no-wait
```

### 비용 최적화 팁

실습 중 비용을 절감하려면:

1. **사용하지 않을 때 일시 중지**
   - Container Apps는 사용하지 않을 때 자동으로 0으로 스케일링됨
   - Azure OpenAI는 사용량 기반 과금

2. **저렴한 SKU 사용**
   - AI Search: Basic tier 대신 Free tier 사용 (개발용)
   - Container Apps: Consumption plan 사용

3. **정기적인 모니터링**
   - Azure Cost Management에서 일일 비용 확인
   - 알림 설정으로 예산 초과 방지

---

### 🎉 완료!

첫 번째 노트북을 성공적으로 완료했습니다. 다음 노트북 **`02_setup_ai_search_rag.ipynb`**에서 RAG 지식 베이스를 구성하는 방법을 배워보세요.